<h3> Face Detection on Dlib DNN</h3>

In [ ]:
# Install dlib library #

# ! pip install dlib
! pip install dlib-binary

In [2]:
import cv2
import dlib
from time import time

# Create face detector object: must not be in function to prevent repeated object creation
cnn_face_detector = dlib.cnn_face_detection_model_v1("./models/mmod_human_face_detector.dat")

'''
This function performs face(s) detection on an image using dlib deep learning based face detector.
Args:
    image:              The input image of the person(s) whose face needs to be detected.
    cnn_face_detector:  The pre-trained dlib deep learning based (CNN) face detection model loaded from
                        the disk required to perform the detection.
    new_width:          The new width of the input image to which it will be resized before passing it to the model.
    display:            A boolean value that is if set to true the function displays the original input image,
                        and the output image with the bounding boxes drawn, confidence scores, and time taken
                        written and returns nothing.
Returns:
    output_image: A copy of input image with the bounding boxes drawn and confidence scores written.
    results:      The output of the face detection process on the input image.
'''

def dlibDnnDetectFaces(image, cnn_face_detector, new_width=600):

    # Get the height and width of the input image.
    height, width, _ = image.shape

    # Calculate the new height of the input image while keeping the aspect ratio constant.
    new_height = int((new_width / width) * height)

    # Resize a copy of input image while keeping the aspect ratio constant.
    resized_image = cv2.resize(image.copy(), (new_width, new_height))

    # Convert the resized image from BGR into RGB format.
    imgRGB = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)

    # Create a copy of the input image to draw bounding boxes and write confidence scores.
    output_image = image.copy()

    # Get the current time before performing face detection.
    start = time()

    # Perform the face detection on the image.
    results = cnn_face_detector(imgRGB, 0)

    # Get the current time after performing face detection.
    end = time()

    # Loop through each face detected in the image.
    for face in results:
        # Retriece the bounding box of the face.
        bbox = face.rect

        # Retrieve the bounding box coordinates and scale them according to the size of original input image.
        x1 = int(bbox.left() * (width / new_width))
        y1 = int(bbox.top() * (height / new_height))
        x2 = int(bbox.right() * (width / new_width))
        y2 = int(bbox.bottom() * (height / new_height))

        # Draw bounding box around the face on the copy of the image using the retrieved coordinates.
        cv2.rectangle(output_image, pt1=(x1, y1), pt2=(x2, y2), color=(0, 255, 0), thickness=width // 200)

        # Draw a filled rectangle near the bounding box of the face.
        # We are doing it to change the background of the confidence score to make it easily visible.
        # cv2.rectangle(output_image, pt1=(x1, y1 - width // 20), pt2=(x1 + width // 16, y1), color=(0, 255, 0),
        #               thickness=-1)

        # Write the confidence score of the face near the bounding box and on the filled rectangle.
        cv2.putText(output_image, text=str(round(face.confidence, 2)), org=(x1, y1 - 25),
                    fontFace=cv2.FONT_HERSHEY_COMPLEX,
                    fontScale=width // 600, color=(255, 255, 255), thickness=width // 200)

    # Return the output image and results of face detection.
    return output_image, results



start_time = 0
capture = cv2.VideoCapture(0)

while (capture.isOpened()):

    success, frame = capture.read()

    # Check if frame is not read properly then continue to the next iteration to read the next frame.
    if not success:
        continue

    # Flip the frame horizontally for natural (selfie-view) visualization.
    frame = cv2.flip(frame, 1)

    # Get the height and width of the frame.
    frame_height, frame_width, _ = frame.shape

    # Perform face detection using the Dlib DNN algorithm.
    frame, _ = dlibDnnDetectFaces(frame, cnn_face_detector)

    # Write the currently selected method on the frame.
    # cv2.putText(frame, (frame_width // 3, frame_height // 8), cv2.FONT_HERSHEY_PLAIN, 4, (255, 155, 0), 3)

    # Set the time for this frame to the current time.
    end_time = time()

    # Check if the difference between the previous and this frame time &gt; 0 to avoid division by zero.
    if (end_time - start_time) > 0:
        # Calculate the number of frames per second.
        frames_per_second = 1.0 / (end_time - start_time)

        # Write the calculated number of frames per second on the frame.
        cv2.putText(frame, 'FPS: {}'.format(int(frames_per_second)), (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0),
                    3)

    # Update the previous frame time to this frame time.
    # As this frame will become previous frame in next iteration.
    start_time = end_time

    cv2.imshow('Haar Cascade', frame)

    if cv2.waitKey(1) == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()
